In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim
from wgan import FE, Discriminator, Classifier, Wasserstein_Loss, Grad_Loss
from tqdm import tqdm
import os
from scipy import io
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

In [2]:
import os
os.getcwd()

'/home/hun/바탕화면/AAI/WGAN'

In [3]:
# path
path = r'../data_preprocessed_matlab/'  # 경로는 저장 파일 경로
file_list = os.listdir(path)

print("data path check")
for i in file_list:    # 확인
    print(i, end=' ')


for i in tqdm(file_list, desc="read data"): 
    mat_file = io.loadmat(path+i)
    data = mat_file['data']
    labels = np.array(mat_file['labels'])
    val = labels.T[0].round().astype(np.int8)
    aro = labels.T[1].round().astype(np.int8)
    
    if(i=="s05.mat"): 
        Data = data
        VAL = val
        ARO = aro
        continue
        
    Data = np.concatenate((Data ,data),axis=0)   # 밑으로 쌓아서 하나로 만듬
    VAL = np.concatenate((VAL ,val),axis=0)
    ARO = np.concatenate((ARO ,aro),axis=0)

data path check
s05.mat s08.mat s17.mat s07.mat s06.mat s27.mat s22.mat s10.mat s25.mat s20.mat s16.mat s15.mat s21.mat s03.mat s02.mat s31.mat s11.mat s12.mat s32.mat s13.mat s29.mat s09.mat s28.mat s01.mat s30.mat s14.mat s19.mat s26.mat s04.mat s18.mat s24.mat s23.mat 

read data: 100%|██████████| 32/32 [00:19<00:00,  1.63it/s]


In [4]:
# eeg preprocessing

eeg_data = []
peripheral_data = []

for i in tqdm(range(len(Data)), desc="preprocess channel"):
    for j in range (40): 
        if(j < 32): # get channels 1 to 32
            eeg_data.append(Data[i][j])
        else:
            peripheral_data.append(Data[i][j])

# set data type, shape
eeg_data = np.reshape(eeg_data, (len(Data),1,32, 8064))
eeg_data = eeg_data.astype('float32')
eeg_data32 = torch.from_numpy(eeg_data)
VAL = (torch.from_numpy(VAL)).type(torch.long)

preprocess channel: 100%|██████████| 1280/1280 [00:00<00:00, 68220.00it/s]


In [5]:
#data 40 x 40 x 8064 video/trial x channel x data
#labels 40 x 4 video/trial x label (valence, arousal, dominance, liking)
#32명 -> 12 / 12 / 8

# data split
print("data split")
train_data, val_data,train_label, val_label = train_test_split(eeg_data32, VAL, test_size=0.25)
x_train, x_test, y_train, y_test = train_test_split(train_data, train_label, test_size=0.5)

# make data loader
print("make data loader")
target_dataset = TensorDataset(x_train, y_train)
source_dataset = TensorDataset(x_test, y_test)
val_dataset = TensorDataset(val_data, val_label)
target_dataloader = DataLoader(target_dataset, 64, shuffle=True)
source_dataloader = DataLoader(source_dataset, 64, shuffle=True)
val_dataloader = DataLoader(val_dataset, 64, shuffle=True)

data split
make data loader


In [6]:
# cuda
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("device: ", device)

#model
dis = Discriminator(15960).to(device)
fe = FE(32).to(device)
classifier = Classifier().to(device)

#optim
optimizer_dis = optim.Adam(dis.parameters(),lr=0.0001,betas=(0,0.9))
optimizer_fe = optim.Adam(fe.parameters(),lr=0.0001, betas=(0,0.0))
optimizer_cls = optim.Adam(classifier.parameters(),lr=0.0001, betas=(0,0.9))

#cls_loss
criterion = nn.CrossEntropyLoss().to(device)

device:  cuda:0


In [7]:
# train WGAN
accuracy_s = []
accuracy_t = []
accuracy_val = []

best_loss = 10000000
limit_epoch = 500
limit_check = 0
val_loss = 0
nb_epochs = 1000
lambda_hyper = 10
mu_hyper = 1
n = 5

log = open('Wasserstein_log.txt', 'w')

torch.autograd.set_detect_anomaly(True)

# while parameter converge
for epoch in tqdm(range(nb_epochs)):
    temp_accuracy_t = 0
    temp_accuracy_s = 0
    temp_accuracy_val = 0
    temp_gloss = 0
    temp_wdloss = 0
    temp_gradloss = 0
    temp_clsloss = 0

    print(epoch+1, ": epoch", file=log)

    temp = 0.0 #batch count
    fe.train()
    dis.train()
    classifier.train()
    # batch
    for i, (target, source) in enumerate(zip(target_dataloader, source_dataloader)):
        temp += 1.0

        x_target = target[0].to(device)
        y_target = target[1].to(device)
        x_source = source[0].to(device)
        y_source = source[1].to(device)

        # update discriminator
        for p in fe.parameters() :
            p.requires_grad = False
        for p in dis.parameters() :
            p.requires_grad = True
        for p in classifier.parameters() :
            p.requires_grad = False
        
        for k in range(n) :
            optimizer_dis.zero_grad()
            wd_grad_loss = 0
            feat_t = fe(x_target)
            feat_s = fe(x_source)
            pred_t = classifier(feat_t)
            pred_s = classifier(feat_s)
            for j in range(feat_s.size(0)) :
                epsil = torch.rand(1).item()
                feat = epsil*feat_s[j,:]+(1-epsil)*feat_t[j,:]
                dc_t = dis(feat_t)
                dc_s = dis(feat_s)
                wd_loss = Wasserstein_Loss(dc_s, dc_t)
                grad_loss = Grad_Loss(feat, dis, device)
                wd_grad_loss = wd_grad_loss - (wd_loss-lambda_hyper*grad_loss)
            wd_grad_loss = wd_grad_loss / feat_s.size(0)
            wd_grad_loss.backward()
            optimizer_dis.step()

        # update classifier
        for p in fe.parameters() :
            p.requires_grad = False
        for p in dis.parameters() :
            p.requires_grad = False
        for p in classifier.parameters() :
            p.requires_grad = True
        
        optimizer_cls.zero_grad()
        feat_s = fe(x_source)
        pred_s = classifier(feat_s)
        cls_loss_source = criterion(pred_s, y_source-1)
        cls_loss_source.backward()
        optimizer_cls.step()
        
        # update Feature Extractor
        for p in fe.parameters() :
            p.requires_grad = True
        for p in dis.parameters() :
            p.requires_grad = False
        for p in classifier.parameters() :
            p.requires_grad = False
        
        optimizer_fe.zero_grad()
        feat_t = fe(x_target)
        feat_s = fe(x_source)
        pred_s = classifier(feat_s)
        dc_t = dis(feat_t)
        dc_s = dis(feat_s)
        wd_loss = Wasserstein_Loss(dc_s, dc_t)
        cls_loss_source = criterion(pred_s, y_source-1)
        fe_loss = cls_loss_source + wd_loss
        fe_loss.backward()
        optimizer_fe.step()
        
        # Temp_Loss
        wd_loss = Wasserstein_Loss(dc_s, dc_t)
        cls_loss_source = criterion(pred_s, y_source-1)
        g_loss = cls_loss_source + mu_hyper*(wd_loss - lambda_hyper*grad_loss)

        feat_t = fe(x_target)
        feat_s = fe(x_source)
        pred_t = classifier(feat_t)
        pred_s = classifier(feat_s)
        
        temp_wdloss = temp_wdloss + wd_loss
        temp_clsloss = temp_clsloss + cls_loss_source
        temp_gloss = temp_gloss + g_loss

        temp_accuracy_t += ((torch.argmax(pred_t,1)+1)== y_target).to(torch.float).mean()
        temp_accuracy_s += ((torch.argmax(pred_s,1)+1)== y_source).to(torch.float).mean()
    
    print("\ngloss", temp_gloss.item()/temp, file=log)
    print("wd_loss", temp_wdloss.item()/temp, file=log)
    print("cls_loss", temp_clsloss.item()/temp, file=log)
    print("acc_t", temp_accuracy_t.item()/temp, file=log)
    print("acc_s", temp_accuracy_s.item()/temp, file=log)
    
    accuracy_t.append(temp_accuracy_t/temp)
    accuracy_s.append(temp_accuracy_s/temp)
    
    fe.eval()
    dis.eval()
    classifier.eval()
    val_loss = 0
    temp = 0
    for x_val, y_val in val_dataloader:
        x_val = x_val.to(device)
        y_val = y_val.to(device)
        pred_val = classifier(fe(x_val))
        temp_accuracy_val += ((torch.argmax(pred_val,1)+1)== y_val).to(torch.float).mean()
        loss = criterion(pred_val, y_val-1)
        val_loss += loss.item() * x_val.size(0)
        temp += 1
    val_total_loss = val_loss / len(val_dataloader.dataset)
    print("val_loss :", val_total_loss, file=log)
    print("acc_val :", temp_accuracy_val.item()/temp, file=log)
    accuracy_val.append(temp_accuracy_val.item()/temp)
    if val_total_loss > best_loss:
        limit_check += 1
        if(limit_check >= limit_epoch):
            break
    else:
        best_loss = val_total_loss
        limit_check = 0
    print()

print("accuracy_t ", sum(accuracy_t)/len(accuracy_t), file=log)
print("accuracy_s ", sum(accuracy_s)/len(accuracy_s), file=log)
print("accuracy_val", sum(accuracy_val)/len(accuracy_val), file=log)
print("best_val_loss ", best_loss, file=log)

log.close()

  0%|          | 0/1000 [00:00<?, ?it/s]

1 : epoch


  0%|          | 1/1000 [00:15<4:23:20, 15.82s/it]


gloss -6.219282150268555
wd_loss 0.4094912111759186
cls_loss 2.165621042251587
acc_t 0.044921875
acc_s 0.099609375
val_loss : 2.243357515335083
acc_val : 0.059375

2 : epoch


  0%|          | 2/1000 [00:31<4:17:17, 15.47s/it]

...

 10%|█         | 102/1000 [25:45<3:46:44, 15.15s/it]


gloss -94006016.0
wd_loss 649.6484985351562
cls_loss 1.480655312538147
acc_t 0.13671875
acc_s 0.701171875
val_loss : 2.2543187618255613
acc_val : 0.146875

103 : epoch


 10%|█         | 102/1000 [26:00<3:49:00, 15.30s/it]


gloss -133394432.0
wd_loss 643.0953369140625
cls_loss 1.478195309638977
acc_t 0.125
acc_s 0.70703125


KeyboardInterrupt: 